In [6]:
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

# Load the dataset and standardize the features
df = pd.read_csv('kc_house_data.csv')
features = df.drop(columns=['id', 'date', 'zipcode', 'price'])
response = df['price']

scaler = StandardScaler()
numeric_cols = features.select_dtypes(include=[np.number]).columns
features[numeric_cols] = scaler.fit_transform(features[numeric_cols])
response = response / 1000

# Mean and standard deviation of the features
stats = pd.DataFrame({
    'Mean': features[numeric_cols].mean().round(6),
    'Std Dev': features[numeric_cols].std(ddof=0).round(6)
})

print(stats.to_string())

               Mean  Std Dev
bedrooms        0.0      1.0
bathrooms      -0.0      1.0
sqft_living     0.0      1.0
sqft_lot        0.0      1.0
floors         -0.0      1.0
waterfront      0.0      1.0
view            0.0      1.0
condition      -0.0      1.0
grade           0.0      1.0
sqft_above      0.0      1.0
sqft_basement   0.0      1.0
yr_built        0.0      1.0
yr_renovated    0.0      1.0
lat             0.0      1.0
long           -0.0      1.0
sqft_living15   0.0      1.0
sqft_lot15     -0.0      1.0


# Problem 2.1

In [7]:
# Split the data into training and testing sets, and fit a linear regression model
X_train, X_test, y_train, y_test = train_test_split(features, response, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

# Predictions on training data
y_pred = model.predict(X_train)

# Coefficients
print("Intercept:", model.intercept_)
print("Coefficients:\n", pd.Series(model.coef_, index=X_train.columns))

# Metrics on training data
print("\nMSE:", mean_squared_error(y_train, y_pred))
print("R²:", r2_score(y_train, y_pred))

# Metrics on test data
y_test_pred = model.predict(X_test)
print("\nTest MSE:", mean_squared_error(y_test, y_test_pred))
print("Test R²:", r2_score(y_test, y_test_pred))

Intercept: 539.5055872959464
Coefficients:
 bedrooms         -30.513682
bathrooms         35.066808
sqft_living       79.938617
sqft_lot           3.507234
floors             0.798518
waterfront        49.152517
view              38.697517
condition         18.732037
grade            112.491578
sqft_above        74.960699
sqft_basement     25.633206
yr_built         -74.370988
yr_renovated       8.863557
lat               77.143288
long             -14.308256
sqft_living15     18.286243
sqft_lot15        -9.025491
dtype: float64

MSE: 39834.25349762571
R²: 0.6951038946870624

Test MSE: 45998.56287706282
Test R²: 0.6957298370207377


# Problem 3

In [8]:
# Add a column of ones for the intercept term
X_train_b = np.column_stack([np.ones(X_train.shape[0]), X_train])
X_test_b = np.column_stack([np.ones(X_test.shape[0]), X_test])

# Closed-form solution
def fit_linear_regression(X, y):
    beta = np.linalg.pinv(X.T @ X) @ X.T @ y
    return beta

# Predict function
def predict(X, beta):
    return X @ beta

# Train the model
beta = fit_linear_regression(X_train_b, y_train)

# Extract intercept and coefficients
print("Intercept:", beta[0])
print("Coefficients:\n", pd.Series(beta[1:], index=X_train.columns))

# Predictions and metrics on training set
y_train_pred = predict(X_train_b, beta)
print("\nTraining MSE:", mean_squared_error(y_train, y_train_pred))
print("Training R²:", r2_score(y_train, y_train_pred))

# Predictions and metrics on testing set
y_test_pred = predict(X_test_b, beta)
print("\nTest MSE:", mean_squared_error(y_test, y_test_pred))
print("Test R²:", r2_score(y_test, y_test_pred))

Intercept: 539.5055872959458
Coefficients:
 bedrooms         -30.513682
bathrooms         35.066808
sqft_living       79.938617
sqft_lot           3.507234
floors             0.798518
waterfront        49.152517
view              38.697517
condition         18.732037
grade            112.491578
sqft_above        74.960699
sqft_basement     25.633206
yr_built         -74.370988
yr_renovated       8.863557
lat               77.143288
long             -14.308256
sqft_living15     18.286243
sqft_lot15        -9.025491
dtype: float64

Training MSE: 39834.2534976257
Training R²: 0.6951038946870625

Test MSE: 45998.56287706284
Test R²: 0.6957298370207374


# Problem 4

In [9]:
# Polynomial regression using closed-form solution

def polynomial_features(X, degree):
    X = np.array(X).reshape(-1, 1)
    return np.hstack([X ** i for i in range(1, degree + 1)])

def fit_linear_regression(X, y):
    beta, _, _, _ = np.linalg.lstsq(X, y, rcond=None)
    return beta

def predict(X, beta):
    return X @ beta

# Use sqft_living as the single feature (unstandardized, from original df)
X_single = df['sqft_living']
y = df['price'].values / 1000

# Split using the same random state
X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(X_single, y, test_size=0.2, random_state=42)

# Train and evaluate for different polynomial degrees
results = []
degrees = [1, 2, 3, 4, 5]

for p in degrees:
    # Create polynomial features and add intercept column
    X_train_poly = polynomial_features(X_train_s, p)
    X_test_poly = polynomial_features(X_test_s, p)

    # Standardize polynomial features
    scaler_poly = StandardScaler()
    X_train_poly = scaler_poly.fit_transform(X_train_poly)
    X_test_poly = scaler_poly.transform(X_test_poly)

    # Add intercept column of ones
    X_train_poly = np.column_stack([np.ones(X_train_poly.shape[0]), X_train_poly])
    X_test_poly = np.column_stack([np.ones(X_test_poly.shape[0]), X_test_poly])

    # Fit model
    beta = fit_linear_regression(X_train_poly, y_train_s)
    
    # Predictions
    y_train_pred = predict(X_train_poly, beta)
    y_test_pred = predict(X_test_poly, beta)
    
    # Metrics
    train_mse = mean_squared_error(y_train_s, y_train_pred)
    train_r2 = r2_score(y_train_s, y_train_pred)
    test_mse = mean_squared_error(y_test_s, y_test_pred)
    test_r2 = r2_score(y_test_s, y_test_pred)
    
    results.append({
        'Degree': p,
        'Train MSE': round(train_mse, 4),
        'Train R²': round(train_r2, 4),
        'Test MSE': round(test_mse, 4),
        'Test R²': round(test_r2, 4)
    })

# Display results as a table
results_df = pd.DataFrame(results)
print(results_df.to_string(index=False))

 Degree  Train MSE  Train R²   Test MSE  Test R²
      1 66319.3478    0.4924 76484.9771   0.4941
      2 58871.8551    0.5494 82113.9312   0.4568
      3 58862.5290    0.5495 83663.5267   0.4466
      4 58818.3044    0.5498 88922.1679   0.4118
      5 58798.9524    0.5499 88307.6469   0.4159


# Problem 5

In [10]:
# Gradient Descent for Linear Regression
def gradient_descent(X, y, alpha, n_iters):
  
    N = X.shape[0]
    theta = np.zeros(X.shape[1])  # Initialize theta to zeros
    history = {}

    for i in range(1, n_iters + 1):
        y_pred = X @ theta
        
        gradient = (1 / N) * (X.T @ (y_pred - y))
       
        theta = theta - alpha * gradient

        # Save theta at specific iterations
        if i in [10, 50, 100]:
            history[i] = theta.copy()

    return theta, history

def predict(X, theta):
    return X @ theta

# Use the same standardized training/testing data
# Add intercept column of ones
X_train_b = np.column_stack([np.ones(X_train.shape[0]), X_train])
X_test_b = np.column_stack([np.ones(X_test.shape[0]), X_test])

# Learning rates and iterations to test
learning_rates = [0.01, 0.1, 0.5]
checkpoints = [10, 50, 100]

# Store results
results = []

for alpha in learning_rates:
    theta, history = gradient_descent(X_train_b, y_train.values, alpha, 100)

    for iters in checkpoints:
        t = history[iters]

        # Predictions
        y_train_pred = predict(X_train_b, t)
        y_test_pred = predict(X_test_b, t)

        # Metrics
        train_mse = mean_squared_error(y_train, y_train_pred)
        train_r2 = r2_score(y_train, y_train_pred)
        test_mse = mean_squared_error(y_test, y_test_pred)
        test_r2 = r2_score(y_test, y_test_pred)

        results.append({
            'Alpha': alpha,
            'Iterations': iters,
            'Train MSE': round(train_mse, 4),
            'Train R²': round(train_r2, 4),
            'Test MSE': round(test_mse, 4),
            'Test R²': round(test_r2, 4)
        })

# Display results
results_df = pd.DataFrame(results)
print(results_df.to_string(index=False))

 Alpha  Iterations    Train MSE      Train R²     Test MSE       Test R²
  0.01          10 3.237865e+05 -1.478300e+00 3.474337e+05 -1.298200e+00
  0.01          50 1.563500e+05 -1.967000e-01 1.684999e+05 -1.146000e-01
  0.01         100 8.256856e+04  3.680000e-01 9.157093e+04  3.943000e-01
  0.10          10 7.865950e+04  3.979000e-01 8.745376e+04  4.215000e-01
  0.10          50 4.000497e+04  6.938000e-01 4.620990e+04  6.943000e-01
  0.10         100 3.984818e+04  6.950000e-01 4.599980e+04  6.957000e-01
  0.50          10 2.213264e+08 -1.693058e+03 2.390183e+08 -1.580052e+03
  0.50          50 7.336029e+22 -5.615084e+17 7.919689e+22 -5.238696e+17
  0.50         100 1.037753e+41 -7.943084e+35 1.120317e+41 -7.410647e+35


# Problem 6

In [11]:

def ridge_gradient_descent(X, y, alpha, n_iters, lam):
    theta = np.zeros(X.shape[1])
    for i in range(1, n_iters + 1):
        y_pred = X @ theta
        reg_term = 2 * lam * theta
        reg_term[0] = 0
        gradient = 2 * (X.T @ (y_pred - y)) + reg_term
        theta = theta - alpha * gradient
    return theta

# Metrics

def mse(X, y, theta):
    y_pred = X @ theta
    return float(np.mean((y_pred - y) ** 2))

def r2(X, y, theta):
    y_pred = X @ theta
    ss_res = float(np.sum((y - y_pred) ** 2))
    ss_tot = float(np.sum((y - np.mean(y)) ** 2))
    return 1 - ss_res / ss_tot

# Simulate Data

np.random.seed(100)
N = 1000

X_raw = np.random.uniform(-2, 2, N)
e = np.random.normal(0, 2, N)
y = 1 + 2 * X_raw + e

# Add bias column
X = np.hstack([np.ones(N).reshape(-1, 1), X_raw.reshape(-1, 1)])

# Fit Linear Regression 

alpha = 0.001
n_iters = 10000

theta_ols, _ = gradient_descent(X, y, alpha, n_iters)
theta_ols = theta_ols.flatten()

results = []

results.append({
    "Method": "Linear Regression",
    "Intercept": theta_ols[0],
    "Slope": theta_ols[1],
    "MSE": mse(X, y, theta_ols),
    "R^2": r2(X, y, theta_ols)
})

# Fit Ridge Regression for different lambdas

lambdas = [1, 10, 100, 1000, 10000]
n_iters_ridge = 50000

for lam in lambdas:
    alpha_ridge = 1 / (4 * (N + lam))
    theta_ridge = ridge_gradient_descent(X, y, alpha_ridge, n_iters_ridge, lam)
    theta_ridge = theta_ridge.flatten()

    results.append({
        "Method": "Ridge (lam={})".format(lam),
        "Intercept": theta_ridge[0],
        "Slope": theta_ridge[1],
        "MSE": mse(X, y, theta_ridge),
        "R^2": r2(X, y, theta_ridge)
    })

df = pd.DataFrame(results).set_index("Method")
print(df.round(4))

                   Intercept   Slope     MSE     R^2
Method                                              
Linear Regression     1.0442  1.8730  4.0092  0.5359
Ridge (lam=1)         1.0443  1.8716  4.0092  0.5359
Ridge (lam=10)        1.0442  1.8589  4.0095  0.5359
Ridge (lam=100)       1.0433  1.7411  4.0322  0.5333
Ridge (lam=1000)      1.0384  1.0656  4.8696  0.4364
Ridge (lam=10000)     1.0322  0.2184  7.6227  0.1177
